# Building a Book Recommender System

In [1]:
import sys
!{sys.executable} -m pip install pandas numpy ipython

In [2]:
import os
os.environ['SKLEARN_NO_OPENMP'] = '1'
!{sys.executable} -m pip install Cython scikit-learn

In [3]:
import pandas as pd
import numpy as np
import re 

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.mode.chained_assignment = None

# Read in and Inspect Data

In [5]:
my_books = pd.read_csv('./goodreads_library_export.csv')
all_books = pd.read_csv('./goodbooks-10k/books.csv')
ratings = pd.read_csv('./goodbooks-10k/ratings.csv')
to_read = pd.read_csv('./goodbooks-10k/to_read.csv')

In [6]:
read_books = my_books[my_books['Read Count'] > 0]

In [7]:
read_books.count()

Book Id                       236
Title                         236
Author                        236
Author l-f                    236
Additional Authors             83
ISBN                          236
ISBN13                        236
My Rating                     236
Average Rating                236
Publisher                     230
Binding                       230
Number of Pages               231
Year Published                231
Original Publication Year     228
Date Read                      82
Date Added                    236
Bookshelves                    17
Bookshelves with positions     17
Exclusive Shelf               236
My Review                       3
Spoiler                         0
Private Notes                   0
Read Count                    236
Recommended For                 0
Recommended By                  0
Owned Copies                  236
Original Purchase Date          0
Original Purchase Location      0
Condition                       0
Condition Desc

In [8]:
read_books.head()

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
1,7723797,Business Model Generation,Alexander Osterwalder,"Osterwalder, Alexander",Yves Pigneur,"=""2839906171""","=""9782839906173""",0,4.18,Alexander Osterwalder & Yves Pigneur,NaN,273.0,2010.0,2010.0,NaN,2019/11/09,currently-reading,currently-reading (#2),currently-reading,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1973951,The Manipulated Man,Esther Vilar,"Vilar, Esther",NaN,"=""0953096424""","=""9780953096428""",0,3.60,Pinter & Martin Ltd,Paperback,154.0,2005.0,1971.0,2019/11/09,2019/11/09,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
20,27036528,Ego Is the Enemy,Ryan Holiday,"Holiday, Ryan",NaN,"=""1591847818""","=""9781591847816""",3,4.14,Portfolio,Hardcover,226.0,2016.0,2016.0,2019/11/02,2017/09/11,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
21,25698,An Inquiry into the Nature and Causes of the W...,Adam Smith,"Smith, Adam",NaN,"=""0553585975""","=""9780553585971""",4,3.88,Arlington House,Leather Bound,1264.0,2003.0,1776.0,2019/11/01,2018/02/22,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23,13629,The Mythical Man-Month: Essays on Software Eng...,Frederick P. Brooks Jr.,"Jr., Frederick P. Brooks",NaN,"=""0201835959""","=""9780201835953""",4,4.05,Addison-Wesley Professional,Paperback,322.0,1995.0,1975.0,NaN,2019/08/20,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [9]:
all_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [10]:
ratings.count()

user_id    5976479
book_id    5976479
rating     5976479
dtype: int64

In [11]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [12]:
to_read.count()

user_id    912705
book_id    912705
dtype: int64

In [13]:
to_read.head()

,user_id,book_id
0,9,8
1,15,398
2,15,275
3,37,7173
4,34,380


In [14]:
# Choose books to include in the recommendation algorithm
my_books['to_recommend'] = my_books.apply(lambda row: 1 if row['Read Count'] == 1 and row['My Rating'] >= 4 else 0, axis=1)

In [15]:
# Filter out unwanted books
unwanted_authors = ['Eoin Colfer', 'C.S. Lewis', 'J.K. Rowling', 'J.R.R. Tolkien', 'Enid Blyton']
my_books['to_recommend'] = my_books.apply(lambda row: 0 if row['Author'] in unwanted_authors else row['to_recommend'], axis=1)

In [16]:
my_books[my_books['to_recommend'] == 1].head()

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID,to_recommend
21,25698,An Inquiry into the Nature and Causes of the W...,Adam Smith,"Smith, Adam",NaN,"=""0553585975""","=""9780553585971""",4,3.88,Arlington House,Leather Bound,1264.0,2003.0,1776.0,2019/11/01,2018/02/22,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
23,13629,The Mythical Man-Month: Essays on Software Eng...,Frederick P. Brooks Jr.,"Jr., Frederick P. Brooks",NaN,"=""0201835959""","=""9780201835953""",4,4.05,Addison-Wesley Professional,Paperback,322.0,1995.0,1975.0,NaN,2019/08/20,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
24,85009,Design Patterns: Elements of Reusable Object-O...,Erich Gamma,"Gamma, Erich","Ralph Johnson, John Vlissides, Richard Helm","=""0201633612""","=""9780201633610""",4,4.17,Addison-Wesley Professional,Hardcover,416.0,1994.0,1994.0,NaN,2017/10/31,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
25,30186948,Think and Grow Rich,Napoleon Hill,"Hill, Napoleon",Ben Holden-Crowther,"=""""","=""""",4,4.18,NaN,Paperback,233.0,2016.0,1937.0,2019/08/05,2018/09/15,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
26,47148164,J'ai vécu l'Everest,Pierre Paperon,"Paperon, Pierre",NaN,"=""225919883X""","=""9782259198837""",4,4.00,Plon,Paperback,260.0,2003.0,NaN,2019/08/05,2019/07/19,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
29,12544648,Cracking the Coding Interview: 150 Programming...,Gayle Laakmann McDowell,"McDowell, Gayle Laakmann",NaN,"=""1466208686""","=""9781466208681""",4,4.35,Createspace,Paperback,500.0,2011.0,2008.0,2019/07/14,2019/06/01,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
30,11468377,"Thinking, Fast and Slow",Daniel Kahneman,"Kahneman, Daniel",NaN,"=""0374275637""","=""9780374275631""",5,4.14,"Farrar, Straus and Giroux",Hardcover,499.0,2011.0,2011.0,2019/07/14,2017/09/11,best,best (#13),read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
44,3869,A Brief History of Time,Stephen Hawking,"Hawking, Stephen",NaN,"=""0553380168""","=""9780553380163""",5,4.17,Bantam Books,Paperback,212.0,1998.0,1988.0,2019/05/11,2017/09/12,best,best (#12),read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
55,36689737,21 Lessons for the 21st Century,Yuval Noah Harari,"Harari, Yuval Noah",NaN,"=""1787330877""","=""9781787330870""",5,4.19,Jonathan Cape,Paperback,368.0,2018.0,2018.0,2019/01/04,2018/12/26,best,best (#11),read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1
57,13589182,Mastery,Robert Greene,"Greene, Robert",NaN,"=""0670024961""","=""9780670024964""",4,4.29,Viking,Hardcover,318.0,2012.0,2012.0,2019/01/04,2018/10/22,NaN,NaN,read,NaN,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1


# Recommendation Algorithm

In [17]:
def book_recommendation_system(my_books = my_books, all_books = all_books, ratings = ratings, to_read = to_read,
                              series = 'Yes', new_author_only = 'No', number_of_similar_quantile = 0.99, english_only = 'No', 
                              rec_weight_avg_rating = 0.5, rec_weight_read_ratio = 0.4, rec_weight_perc_4_5 = 0.1, 
                              rec_weight_perc_1_2 = -0.1, rec_weight_author_previous = 0.1, return_dataset = 'Yes',
                              num_similar_ratings = 50):
    
    """
    An algorithm to predict the books that I should read next based on what I've read to-date. The data is limited to the
    top 10,000 books on Goodreads. The 'Series' toggle allows you to determine whether you want a series or not; the 
    'new_author' toggle allows you to filter for new or previous authors; 'english_only' filters for only English
    language books; If you want the full data set, toggle 'return_dataset' to 'Yes'.
    
    To use with your own data, you would need to export your data, create a "to_recommend" column where you decide which 
    books you want to be used with the recommendor. If all, create a "to_recommend" column and put a '1' for all values.
    
    """
    
    print('Getting recommendations...')
    
    # Filter out books that I haven't yet read or am currently reading

    my_books = my_books[~my_books['Exclusive Shelf'].isin(['to read','currently reading'])]
    
    # Extract relevant columns from my books

    my_books = my_books[['Book Id', 'Title','Author','ISBN','ISBN13','My Rating','Average Rating',
                     'Publisher','Number of Pages','Year Published','to_recommend']]
    
    # Rename columns to python-friendly format

    my_books = my_books.rename(columns = {'Book Id':'book_id', 'My Rating':'my_rating','Average Rating':'average_rating',
                          'Number of Pages':'num_pages','Year Published':'year_published'})
    
    # Drop irrelevant columns from all books

    all_books = all_books.drop(['image_url','small_image_url'], axis=1)
    
    # Standardize any english-language book and filter for only english

    all_books.language_code = all_books.language_code.replace(to_replace = ['en-US','en-CA','en-GB','en'], 
                                                          value = ['eng','eng','eng','eng'])
    
    if english_only == 'Yes':
        
        all_books = all_books[all_books.language_code == 'eng']
    
    # Add a column to all books indicating if I've read the book

    all_books['jordan_read'] = [1 if i else 0 for i in all_books.best_book_id.isin(my_books.book_id)]
    all_books['to_recommend'] = [1 if i else 0 for i in all_books.best_book_id.\
                                 isin(my_books[my_books.to_recommend == 1].book_id)]
    
    # Add my rating to the books that I've read

    all_books = pd.merge(all_books, my_books[['book_id','my_rating']], left_on = 'best_book_id', right_on = 'book_id', 
                         how = 'left', copy = False)
    
    # Drop overlapping columns
    
    all_books.drop(['book_id_y'], axis=1, inplace=True)
    all_books.rename(columns = {'book_id_x':'book_id'}, inplace=True)
    
    # Merge addtional columns from my books that didn't match on book_id
    
    all_books = pd.merge(all_books, my_books[['Title','Author','my_rating','to_recommend']], left_on = ['title','authors'], 
                     right_on = ['Title','Author'], how = 'left')
    
    # Ensure that the ratings and other information carries over if no match on book_id
    
    all_books['my_rating_x'] = [j if pd.notnull(j) else all_books.loc[i,'my_rating_y'] for i,j in 
                                all_books['my_rating_x'].items()]
    all_books['to_recommend_x'] = [j if j == 1 else all_books.loc[i, 'to_recommend_y'] for 
                                   i, j in all_books.to_recommend_x.items()]
    all_books['to_recommend_x'] = [i if pd.notnull(i) else 0 for i in all_books.to_recommend_x]

    all_books['jordan_read'] = [1 if pd.notnull(all_books.loc[i,'my_rating_x']) else 0 for i in range(len(all_books))]
    
    # Drop columns that aren't relevant and rename others
    
    all_books = all_books.drop(['Title','Author','my_rating_y','to_recommend_y'], axis = 1).rename(
    columns = {'to_recommend_x':'to_recommend', 'my_rating_x':'my_rating'})
    
    # Match to ratings on only the books that I rated higher than a 3
    
    ratings = pd.merge(ratings, all_books[all_books.my_rating > 3][['book_id','my_rating','to_recommend']], left_on = 'book_id', 
                   right_on = 'book_id', how = 'left')
    
    # Identify the most similar users by grouping on user ID and summing the books that I want recommended
    
    most_similar_readers = ratings.groupby(by='user_id').agg({'to_recommend':'sum','my_rating':'mean'}).\
                           sort_values(by='to_recommend', ascending=False)

    # Keep only the readers that are higher than the median value
    
    most_similar_readers = most_similar_readers[most_similar_readers.to_recommend >= most_similar_readers.\
                            to_recommend.quantile(number_of_similar_quantile )].sort_values(by='to_recommend', ascending=False)
    
    print('\nWow! There are %i similar readers to you...' %len(most_similar_readers))
    
    # Isolate only the ratings of my most similar readers
    
    most_similar_by_book = ratings[ratings.user_id.isin(most_similar_readers.index)].groupby(by='book_id').agg(
    {'user_id':'count','rating':'mean'}).sort_values(by='rating', ascending=False)
    
    print('\n...who have read a total of %i books!'%most_similar_by_book.user_id.sum())

    # Rename columns to be more descriptive
    
    most_similar_by_book = most_similar_by_book.rename(columns={'user_id':'num_similar_ratings',
                                                            'rating':'avg_rating_similar_users'})
    
    # Identify the books marked as "to-read" by similar users
    
    most_to_read = to_read[to_read.user_id.isin(most_similar_readers.index)].groupby(by='book_id').count().sort_values(by=
                                                                                                'user_id',ascending=False)

    most_to_read = most_to_read.rename(columns={'user_id':'number_to_read'})
    
    # Create a new data frame to merge the number of reads and ratings by similar users
    
    all_books_rec = pd.merge(all_books, most_similar_by_book, left_on = 'book_id', right_index=True, how = 'left')
    all_books_rec = pd.merge(all_books_rec, most_to_read, left_on='book_id', right_index=True, how = 'left')
    
    # Fill in missing values for number to read
    
    all_books_rec['number_to_read'] = all_books_rec['number_to_read'].fillna(0)
    
    # Create a new column that has the number of ratings plus "to read" for each book
    
    all_books_rec['rated_plus_to_read'] = all_books_rec.num_similar_ratings + all_books_rec.number_to_read

    print("\nThe most read new book is: %s" % all_books_rec[all_books_rec.jordan_read == 0].sort_values(by='num_similar_ratings', 
                                                                    ascending = False)[:1]['title'].iloc[0])
    
    # Create a ratio of rated plus to read to total reads in the data set
    
    all_books_rec['rated_plus_to_read_ratio'] = all_books_rec.rated_plus_to_read / all_books_rec['work_ratings_count']
    
    # Create individual ratios for each variable
    
    all_books_rec['rated_ratio'] = all_books_rec.num_similar_ratings / all_books_rec['work_ratings_count']
    all_books_rec['to_read_ratio'] = all_books_rec.number_to_read / all_books_rec['work_ratings_count']
    
    def percent_rating(rating_number, df = all_books_rec):
        """
        Returns the percentage of ratings for each value
        """
        return df['ratings_%s'%rating_number] / df.work_ratings_count
    
    all_books_rec['percent_1'] = percent_rating(1)
    all_books_rec['percent_2'] = percent_rating(2)
    all_books_rec['percent_3'] = percent_rating(3)
    all_books_rec['percent_4'] = percent_rating(4)
    all_books_rec['percent_5'] = percent_rating(5)

    # Create a variable for very high and very low ratings
    
    all_books_rec['percent_4_5'] = all_books_rec.percent_4 + all_books_rec.percent_5
    all_books_rec['percent_1_2'] = all_books_rec.percent_1 + all_books_rec.percent_2
    
    # Add a column flag on whether the author has been read previously or not
    
    all_books_rec['author_read_previously'] = [1 if i else 0 for i in all_books_rec.authors.isin(my_books.Author)]
    
    # Pull only the relevant recommendation columns
    
    all_books_rec_cols = ['authors','title','language_code','jordan_read', 'to_recommend', 'my_rating','num_similar_ratings',
                     'avg_rating_similar_users','rated_plus_to_read_ratio','author_read_previously','percent_1_2',
                     'percent_4_5']

    recs_data = all_books_rec[all_books_rec_cols]
    
    # Standardize columns between 0 and 1 to offset different scales

    from sklearn.preprocessing import MinMaxScaler

    cols_to_standardize = ['avg_rating_similar_users','rated_plus_to_read_ratio','percent_1_2','percent_4_5']
    
    
    for col in cols_to_standardize:

        min_max_scaler = MinMaxScaler()

        recs_data.loc[:,col] = min_max_scaler.fit_transform(np.array(recs_data.loc[:,col]).reshape(-1,1))
        
    # Create recommendation weights based on user inputs
    
    recommendation_weights = {'avg_rating_similar_users':rec_weight_avg_rating,
                         'rated_plus_to_read_ratio':rec_weight_read_ratio,
                         'percent_4_5':rec_weight_perc_4_5,
                          'percent_1_2':rec_weight_perc_1_2,
                         'author_read_previously':rec_weight_author_previous}
    
    # Create the weighted value of each row, store it in a list, then add that list to the data frame
    
    rec_weights = []
    
    for i in range(len(recs_data)):

        total_val = 0

        for j in cols_to_standardize:

            total_val += recommendation_weights.get(j) * recs_data.loc[i, j]

        rec_weights.append(total_val)

    recs_data.loc[:,'recommendation_weights'] = rec_weights
    
    # Eliminate series if toggle set to "No"
    
    if series == 'No':
        
        recs_data = recs_data[~(recs_data.title.str.contains('#'))]
        
    # Include only new authors if toggle set to "Yes"
    
    if new_author_only == 'Yes':
        
        recs_data = recs_data[recs_data.author_read_previously == 0]
    
    # Filter for only books that I haven't read
    
    recs_data = recs_data[recs_data.jordan_read == 0]
    
    
    # If a series is in the recommendation data set, see whether I've read any books in the series and if yes, remove
    # any values that had an average rating below 4
    
    for i in recs_data.title:
        
        if '#' in i:
            
            try:
                relevant_books = my_books[my_books.Title.str.contains(i.split('(')[1].split(',')[0])]
                            
                average_relevant_books = relevant_books['my_rating'].mean()
            
                if average_relevant_books < 4:
                
                    recs_data = recs_data[~recs_data.title.str.contains(i.split('(')[1].split(',')[0])]
                    
            except:
                
                continue
    
    # Remove any "Boxed Set" books as they don't represent a single book
    
    recs_data = recs_data[~(recs_data.title.str.contains('Boxed Set') | recs_data.title.str.contains('Boxset'))]
    
    # Get the top recommendations by total score, popularity, and rating for books that have more than the values
    # set by the user
    
    top_rec_by_weight = recs_data[recs_data.num_similar_ratings > num_similar_ratings].sort_values(by='recommendation_weights', 
                                                                                   ascending=False)[:1]
    
    top_rec_by_popularity = recs_data[recs_data.num_similar_ratings > num_similar_ratings].sort_values(by = 'rated_plus_to_read_ratio', 
                                                                                       ascending=False)[:1]
    
    top_rec_by_rating = recs_data[recs_data.num_similar_ratings > num_similar_ratings].sort_values(by='avg_rating_similar_users', 
                                                                                   ascending=False)[:1]
    
    
    print('\nIf you want to read the highest rated book by your similar users, you should read %s by %s' %(
        top_rec_by_rating['title'].iloc[0], top_rec_by_rating['authors'].iloc[0]))
    
    print('\nTaking into account all factors, we recommend that you should read %s by %s!' %(
        top_rec_by_popularity['title'].iloc[0], top_rec_by_popularity['authors'].iloc[0]))
    
    # Returns the full data set of recommendation values if set by the user
    
    if return_dataset == 'Yes':
        
        return recs_data    

In [ ]:
book_recommendation_system()

Getting recommendations...

Wow! There are 879 similar readers to you...

...who have read a total of 119030 books!

The most read new book is: Lord of the Flies

If you want to read the highest rated book by your similar users, you should read The Hobbit: Graphic Novel by Chuck Dixon, J.R.R. Tolkien, David Wenzel, Sean Deming

Taking into account all factors, we recommend that you should read The Last of the Mohicans (The Leatherstocking Tales #2) by James Fenimore Cooper!


In [ ]:
book_recommendation_system(series = 'No')

In [ ]:
book_recommendation_system(new_author_only = 'Yes')

In [ ]:
book_recommendation_system(number_of_similar_quantile = 0.95)

In [27]:
recommended = book_recommendation_system(new_author_only = 'No', number_of_similar_quantile = 0.95, rec_weight_avg_rating = 0.8, 
                           rec_weight_read_ratio = 0.2, rec_weight_perc_4_5 = 0.1, rec_weight_perc_1_2 = -0.1, 
                           rec_weight_author_previous = 0)

Getting recommendations...

Wow! There are 2991 similar readers to you...

...who have read a total of 389922 books!

The most read new book is: The Great Gatsby

If you want to read the highest rated book by your similar users, you should read The Complete Calvin and Hobbes by Bill Watterson

Taking into account all factors, we recommend that you should read Mutiny on the Bounty (The Bounty Trilogy, #1) by Charles Bernard Nordhoff, James Norman Hall!


In [28]:
recommended.head()

,authors,title,language_code,jordan_read,to_recommend,my_rating,num_similar_ratings,avg_rating_similar_users,rated_plus_to_read_ratio,author_read_previously,percent_1_2,percent_4_5,recommendation_weights
4,F. Scott Fitzgerald,The Great Gatsby,eng,0,0.0,NaN,1883.0,0.698885,0.123349,0,0.180380,0.629389,0.628678
5,John Green,The Fault in Our Stars,eng,0,0.0,NaN,300.0,0.756667,0.024869,0,0.097064,0.797327,0.680333
6,J.R.R. Tolkien,The Hobbit,eng,0,0.0,NaN,1758.0,0.801195,0.143978,1,0.095473,0.799396,0.740144
7,J.D. Salinger,The Catcher in the Rye,eng,0,0.0,NaN,1827.0,0.688150,0.158270,0,0.247535,0.587681,0.616189
9,Jane Austen,Pride and Prejudice,eng,0,0.0,NaN,1367.0,0.741587,0.113218,0,0.111057,0.790347,0.683843


In [29]:
recommended.count()

authors                     8411
title                       8411
language_code               7490
jordan_read                 8411
to_recommend                8411
my_rating                      0
num_similar_ratings         7493
avg_rating_similar_users    7493
rated_plus_to_read_ratio    7493
author_read_previously      8411
percent_1_2                 8411
percent_4_5                 8411
recommendation_weights      7493
dtype: int64

In [35]:
recommended[recommended.num_similar_ratings > 500].sort_values(by='recommendation_weights', ascending=False).head(15)

,authors,title,language_code,jordan_read,to_recommend,my_rating,num_similar_ratings,avg_rating_similar_users,rated_plus_to_read_ratio,author_read_previously,percent_1_2,percent_4_5,recommendation_weights
188,J.R.R. Tolkien,"The Lord of the Rings (The Lord of the Rings, ...",eng,0,0.0,NaN,786.0,0.857824,0.327364,1,0.053531,0.886290,0.835008
134,George R.R. Martin,"A Storm of Swords (A Song of Ice and Fire, #3)",eng,0,0.0,NaN,716.0,0.866620,0.246169,1,0.015003,0.934795,0.834509
596,"Miguel de Cervantes Saavedra, Roberto González...",Don Quixote,eng,0,0.0,NaN,619.0,0.735057,0.781057,0,0.187941,0.597203,0.785183
128,Ken Kesey,One Flew Over the Cuckoo's Nest,eng,0,0.0,NaN,974.0,0.797485,0.370523,0,0.070204,0.789482,0.784020
102,"Alexandre Dumas, Robin Buss",The Count of Monte Cristo,eng,0,0.0,NaN,1075.0,0.806047,0.346449,0,0.093202,0.777727,0.782580
84,Shel Silverstein,The Giving Tree,eng,0,0.0,NaN,640.0,0.824609,0.163902,0,0.085555,0.836838,0.767596
88,William Goldman,The Princess Bride,eng,0,0.0,NaN,576.0,0.822049,0.180318,0,0.089187,0.798409,0.764625
156,"Dr. Seuss, לאה נאור",Green Eggs and Ham,eng,0,0.0,NaN,512.0,0.813965,0.200081,0,0.077077,0.794824,0.762963
125,Frank Herbert,Dune (Dune Chronicles #1),eng,0,0.0,NaN,841.0,0.790725,0.308219,0,0.118865,0.770784,0.759416
275,Isaac Asimov,Foundation (Foundation #1),eng,0,0.0,NaN,507.0,0.781065,0.350282,1,0.117176,0.749435,0.758134


In [36]:
recommended[recommended.num_similar_ratings > 500].sort_values(by='avg_rating_similar_users', ascending=False).head(15)

,authors,title,language_code,jordan_read,to_recommend,my_rating,num_similar_ratings,avg_rating_similar_users,rated_plus_to_read_ratio,author_read_previously,percent_1_2,percent_4_5,recommendation_weights
134,George R.R. Martin,"A Storm of Swords (A Song of Ice and Fire, #3)",eng,0,0.0,NaN,716.0,0.866620,0.246169,1,0.015003,0.934795,0.834509
188,J.R.R. Tolkien,"The Lord of the Rings (The Lord of the Rings, ...",eng,0,0.0,NaN,786.0,0.857824,0.327364,1,0.053531,0.886290,0.835008
84,Shel Silverstein,The Giving Tree,eng,0,0.0,NaN,640.0,0.824609,0.163902,0,0.085555,0.836838,0.767596
88,William Goldman,The Princess Bride,eng,0,0.0,NaN,576.0,0.822049,0.180318,0,0.089187,0.798409,0.764625
156,"Dr. Seuss, לאה נאור",Green Eggs and Ham,eng,0,0.0,NaN,512.0,0.813965,0.200081,0,0.077077,0.794824,0.762963
49,Shel Silverstein,Where the Sidewalk Ends,eng,0,0.0,NaN,745.0,0.810403,0.131714,0,0.077881,0.806041,0.747481
30,Kathryn Stockett,The Help,eng,0,0.0,NaN,533.0,0.809099,0.066345,0,0.033566,0.902659,0.747458
86,"Elie Wiesel, Marion Wiesel",Night (The Night Trilogy #1),eng,0,0.0,NaN,603.0,0.806385,0.165757,0,0.061190,0.827640,0.754904
102,"Alexandre Dumas, Robin Buss",The Count of Monte Cristo,eng,0,0.0,NaN,1075.0,0.806047,0.346449,0,0.093202,0.777727,0.782580
6,J.R.R. Tolkien,The Hobbit,eng,0,0.0,NaN,1758.0,0.801195,0.143978,1,0.095473,0.799396,0.740144
